# Transfer Learning

В этом ноутбуке будет показана возможность использовать предобученные классификаторы для дальнейшего простого и быстрого их дообучения на нужной базе

### Зачем это нужно?
* Конечная база данных недостаточно представительна для полноценного обучения (сотни или даже десятки картинок)
* Имеется предобученная СНС на представительной БД (например, ImageNet)
* Нужно быстро получить классификатор на новой (конечной) БД

### Как это обычно делают в Keras?
* Берут предобученный СНС-классификатор
* Отрезают все слои, следующие за последним сверточным (Flatten, Dense, Softmax)
* Замораживают веса оставшихся слоев (опционально - для ускорения обучения)
* Сверху навешивают новую "голову" с нужным числом классов и своей функцией потерь
* Собирают (компилируют) новую модель и дообучают

### План
* Обучим сеть для MNIST
* Обучим сеть для Fashion MNIST 
* Сделаем Transfer Learning:
  * для MNIST -> Fashion MNIST
  * для Fashion MNIST -> MNIST
* Увидим, что использование этой техники даже в таком игрушечном примере может улучшать качество по сравнению с обычным обучением
* Покажем, как делать Transfer Learning для предобученных моделей на Keras 

In [1]:
import os
import numpy as np
import random
import tensorflow as tf

# for reproducibility
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)  
tf.random.set_seed(seed)

import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.layers import Conv2D
 
import matplotlib.pyplot as plt

In [2]:
n_epoch_orig = 1
n_epoch_fn = 2

#### Загружаем стандартные интерфейсы для работы с (Fashion) MNIST

In [3]:
from keras.datasets import mnist
from keras.datasets import fashion_mnist

### MNIST

#### Загружаем саму базу MNIST

In [4]:
(m_x_train, m_y_train), (m_x_test, m_y_test) = mnist.load_data()

In [ ]:
print(m_x_train.dtype)

In [ ]:
print(m_x_train.shape, m_x_test.shape, m_y_train.shape, m_y_test.shape)

In [ ]:
ind = 10
m_image = m_x_train[ind]
plt.imshow(m_image)
plt.show()
print(m_y_train[ind])

#### Preprocessing

In [8]:
m_x_train = m_x_train.astype(np.float32)[..., np.newaxis]/255
m_x_test = m_x_test.astype(np.float32)[..., np.newaxis]/255
m_y_train = np_utils.to_categorical(m_y_train, num_classes=10)
m_y_test = np_utils.to_categorical(m_y_test, num_classes=10)

In [ ]:
print(m_x_train.shape, m_x_test.shape, m_y_train.shape, m_y_test.shape)

#### Конструируем простую СНС

In [10]:
m_input_image = Input(shape=(28, 28, 1), name='m_input')
m_conv1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(2,2), padding="same", activation='relu', data_format='channels_last', name='m_conv1')(m_input_image)
m_conv2 = Conv2D(filters=32, kernel_size=(3, 3), strides=(2,2), padding="same", activation='relu', data_format='channels_last', name='m_conv2')(m_conv1)
m_flatten = Flatten()(m_conv2)
m_dense1 = Dense(128, activation='relu')(m_flatten)
m_dense2 = Dense(10, activation='softmax')(m_dense1)
m_model = Model(inputs=m_input_image, outputs=m_dense2)

#### Смотрим краткую инфу по СНС

In [ ]:
m_model.summary()

#### Объединяем с функцией потерь, оптимизатором и набором метрик

In [12]:
m_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### Обучаем

In [ ]:
m_history = m_model.fit(m_x_train, m_y_train, validation_split=0.25, batch_size=32, epochs=n_epoch_orig, verbose=1)

In [14]:
# Plot training & validation accuracy values
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('Model accuracy')
#plt.ylabel('Accuracy')
#plt.xlabel('Epoch')
#plt.legend(['Train', 'Val'], loc='upper left')
#plt.show()

#### Смотрим качество на тесте

In [ ]:
m_score = m_model.evaluate(m_x_test, m_y_test, verbose=0)
print('MNIST acc: ', 100*m_score[1])

#### Сохраняем веса последнего сверточного слоя 'm_conv2' 
* Обращаем внимание, как обращаться к слою по имени (который можно посмотреть в model.summary() )
* Обращаем внимание, как вытаскивать обучаемые веса слоя через **get_weights**() )
* m_c[0] - ядра сверток, m_c[1] - их сдвиги

In [ ]:
m_c = m_model.get_layer('m_conv2').get_weights()
print(m_c[1]) 

### Fashion MNIST

Делаем все то же самое, что и для MNIST

In [17]:
(f_x_train, f_y_train), (f_x_test, f_y_test) = fashion_mnist.load_data()

In [ ]:
ind = 20
f_image = f_x_train[ind]
plt.imshow(f_image)
plt.show()
print(f_y_train[ind])

In [19]:
f_x_train = f_x_train.astype(np.float32)[..., np.newaxis]/255
f_x_test = f_x_test.astype(np.float32)[..., np.newaxis]/255
f_y_train = np_utils.to_categorical(f_y_train, num_classes=10)
f_y_test = np_utils.to_categorical(f_y_test, num_classes=10)

In [20]:
f_input_image = Input(shape=(28, 28, 1), name='f_input')
f_conv1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(2,2), padding="same", activation='relu', data_format='channels_last', name='f_conv1')(f_input_image)
f_conv2 = Conv2D(filters=32, kernel_size=(3, 3), strides=(2,2), padding="same", activation='relu', data_format='channels_last', name='f_conv2')(f_conv1)
f_flatten = Flatten()(f_conv2)
f_dense1 = Dense(128, activation='relu')(f_flatten)
f_dense2 = Dense(10, activation='softmax')(f_dense1)
f_model = Model(inputs=f_input_image, outputs=f_dense2)

In [ ]:
f_model.summary()

In [22]:
f_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
f_history = f_model.fit(f_x_train, f_y_train, validation_split=0.25, batch_size=32, epochs=n_epoch_orig, verbose=1)

In [24]:
# Plot training & validation accuracy values
#plt.plot(fhistory.history['accuracy'])
#plt.plot(fhistory.history['val_accuracy'])
#plt.title('Model accuracy')
#plt.ylabel('Accuracy')
#plt.xlabel('Epoch')
#plt.legend(['Train', 'Val'], loc='upper left')
#plt.show()

In [ ]:
f_score = f_model.evaluate(f_x_test, f_y_test, verbose=0)
print('Fashion MNIST: ', 100*f_score[1])

In [ ]:
f_c = f_model.get_layer('f_conv2').get_weights()
print(f_c[1])

### MNIST -> Fashion MNIST  

#### Скопируем архитектуру модели MNIST (clone_model) и ее веса (set/get weights), чтобы не повредить изначальную модель

In [ ]:
# скопируем модель
f2m_model = keras.models.clone_model(m_model)
f2m_model.set_weights(m_model.get_weights())
f2m_model.summary()

#### Проверим, что веса скопировались

In [ ]:
f2m_c = f2m_model.get_layer('m_conv2').get_weights()
print('Conv the same: ', np.all(f2m_c[0] == m_c[0]), 'Biases the same: ', np.all(f2m_c[1] == m_c[1]))

#### Убираем последние несверточные слои
* **Внимание**: раньше можно было обращаться к .layers, теперь же приходится напрямую к ._layers !

In [ ]:
f2m_model._layers[-3:] = []
f2m_model.summary()

#### Замораживаем сверточные слои из модели MNIST

In [30]:
for layer in f2m_model.layers:
    layer.trainable = False

#### Приделываем новую классификационную "голову"
* Для этого на вход новому слою Flatten подаем выход последнего сверточного слоя из новой модели

In [31]:
new_flatten = Flatten()(f2m_model.get_layer('m_conv2').output)
new_dense1 = Dense(128, activation='relu')(new_flatten)
new_dense2 = Dense(10, activation='softmax')(new_dense1)

new_model = Model(inputs=f2m_model.inputs, outputs=new_dense2)

In [32]:
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### Проверяем, что после построения новой модели веса не поменялись

In [ ]:
new_c = new_model.get_layer('m_conv2').get_weights()
print('Conv the same: ', np.all(new_c[0] == m_c[0]), 'Biases the same: ', np.all(new_c[1] == m_c[1]))

#### Дообучаем новую модель на Fashion MNIST

In [ ]:
new_history = new_model.fit(f_x_train, f_y_train, validation_split=0.25, batch_size=32, epochs=n_epoch_fn, verbose=1)

#### На всякий случай еще раз проверим, что сверточные веса были заморожены

In [ ]:
new2_c = new_model.get_layer('m_conv2').get_weights()
print('Conv the same: ', np.all(new2_c[0] == m_c[0]), 'Biases the same: ', np.all(new2_c[1] == m_c[1]))

#### Сравним качество дообученной модели с исходной

In [ ]:
new_score = new_model.evaluate(f_x_test, f_y_test, verbose=0)
print('Fashion model acc =', 100*f_score[1], 'MNIST->Fashion model acc =', 100*new_score[1])

### Fashion MNIST -> MNIST   
* Проделаем все то же самое, только в обратном порядке

In [ ]:
# скопируем модель
m2f_model = keras.models.clone_model(f_model)
m2f_model.set_weights(f_model.get_weights())
m2f_model.summary()

m2f_c = m2f_model.get_layer('f_conv2').get_weights()
print('Conv the same: ', np.all(m2f_c[0] == f_c[0]), 'Biases the same: ', np.all(m2f_c[1] == f_c[1]))

m2f_model._layers[-3:] = []
m2f_model.summary()

for layer in m2f_model.layers:
    layer.trainable = False

new_flatten = Flatten()(m2f_model.get_layer('f_conv2').output)
new_dense1 = Dense(128, activation='relu')(new_flatten)
new_dense2 = Dense(10, activation='softmax')(new_dense1)

new_model = Model(inputs=m2f_model.inputs, outputs=new_dense2)

new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

new_c = new_model.get_layer('f_conv2').get_weights()
print('Conv the same: ', np.all(new_c[0] == f_c[0]), 'Biases the same: ', np.all(new_c[1] == f_c[1]))

new_history = new_model.fit(m_x_train, m_y_train, validation_split=0.25, batch_size=32, epochs=n_epoch_fn, verbose=1)

new2_c = new_model.get_layer('f_conv2').get_weights()
print('Conv the same: ', np.all(new2_c[0] == f_c[0]), 'Biases the same: ', np.all(new2_c[1] == f_c[1]))

new_score = new_model.evaluate(m_x_test, m_y_test, verbose=0)
print('MNIST model acc =', 100*m_score[1], 'Fashion->MNIST model acc =', 100*new_score[1])

## Работа с предобученными моделями в Keras

Для примера возьмем легкую MobileNet, все же доступные предобученные модели можно посмотреть здесь - https://keras.io/applications/

In [38]:
from keras.applications.mobilenet import MobileNet
mobile_net = MobileNet(input_shape=(224,224,3), include_top=False, weights='imagenet', classes=1000)

#### Важные параметры
* input_shape - (224,224,3) или (3, 224, 224) в зависимости от того, когда у вас идут каналы изображения (channels_last или channels_first)
* include_top=False - самый важный параметр, говорит о том, что нужно отрезать всю "голову" СНС после последней свертки
* weights='imagenet' - подгружаем веса СНС, обученной на ImageNet (classes не задаем)
* Если же хотим просто использовать архитектуру без весов, то weights=None и classes=нужное_количество

In [ ]:
mobile_net.summary()

#### Заморозка слоев

In [40]:
for layer in mobile_net.layers:
    layer.trainable = False

#### Приделывание новой "головы"

In [41]:
x = mobile_net.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(111, activation="softmax")(x)

#### Завершение создания новой модели

In [42]:
# creating the final model 
model_final = Model(inputs = mobile_net.inputs, outputs = predictions)
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

#### Ну и дальше обучение model_final ...